In [1]:
import pandas as pd# This imports the pandas library, which is used for handling and analyzing data in table format (like Excel sheets)
import ast # This imports the 'ast' module, which helps convert strings into Python objects (like lists, dictionaries, etc.) safely

In [5]:
# Load the data from CSV files
df_scheme_mapping = pd.read_csv("Top_Optimized_Schemes_with_LP.csv")
df_recommendations = pd.read_csv("Partner_Product_Recommendations.csv")

In [6]:
def safe_eval(val):
    # If the value is a string and looks like a list (starts with '['), try converting it to a real Python list
    try:
        # If there's an error during conversion (e.g. invalid format), just return the original value
        return ast.literal_eval(val) if isinstance(val, str) and val.startswith("[") else val
    except:
        return val

In [11]:
# Convert the 'Partner_id' column values from string representations of lists to actual Python lists (if needed)
df_scheme_mapping["Partner_id"] = df_scheme_mapping["Partner_id"].apply(safe_eval)
# Convert 'Recommended_Products' column values from strings like "['ProdA', 'ProdB']" to real Python lists
df_recommendations["Recommended_Products"] = df_recommendations["Recommended_Products"].apply(safe_eval)
# Convert 'Similarity_Scores' column values from strings like "[0.95, 0.87]" to real lists of numbers
df_recommendations["Similarity_Scores"] = df_recommendations["Similarity_Scores"].apply(safe_eval)

# Create an empty list to store the final results
results = []
# Loop through each row in the recommendations DataFrame
for _, row in df_recommendations.iterrows():
    partner_id = row["Partner_id"]
    # Loop through each recommended product and its corresponding similarity score
    for product, score in zip(row["Recommended_Products"], row["Similarity_Scores"]):
        # Find the corresponding schemes for this product in the scheme mapping DataFrame
        # If schemes are found, extract them and fill missing values with "Not Available"
        # If no schemes found, mark all as "Not Available"
        schemes = df_scheme_mapping[df_scheme_mapping["Product_id"] == product][["Scheme_1", "Scheme_2", "Scheme_3"]]
        scheme_1, scheme_2, scheme_3 = schemes.iloc[0].fillna("Not Available").values if not schemes.empty else ("Not Available", "Not Available", "Not Available")
        
        # Add the combined result to the list
        results.append([partner_id, product, score, scheme_1, scheme_2, scheme_3]) 

# Convert the final results into a DataFrame
df_final_schemes = pd.DataFrame(results, columns=["Partner_id", "Product_id", "Similarity_Scores", "Scheme_1", "Scheme_2", "Scheme_3"])
df_final_schemes.to_csv("Final_Partner_Product_Schemes.csv", index=False)

# Save the final DataFrame to a CSV file
print("Final Partner Product Schemes saved successfully!")

Final Partner Product Schemes saved successfully!
